# Załadowanie wewnętrznych danych
Pierwszym krokiem jest załadowanie zewnętrznych danych, aby utworzyć RDD (Resilient Distributed Dataset). Jest to abstrakcyjny typ danych używany w Apache Spark jako podstawowy.

In [2]:
meteo_rdd = sc.textFile("/FileStore/tables/5q1of39g1497426807924/data_csv-1aa57.gz")
print('Number of raw records: {}'.format(meteo_rdd.count()))

# podziel kazda linie na poszczegolne pola 
splitted_lines = meteo_rdd.map(lambda l: l.split(";"))
def group_height(h):
  if h <= 250:
    return(1)
  elif h > 250 and h <= 1000:
    return(2)
  elif h > 1000:
    return(int(h / 5000) + 3)
  else:
    return(0)

# zrzutuj podzielone pola na odpowiednie typy i stworz krotke (tuple)
measures = splitted_lines.map(lambda p: (int("20" + p[0][:2]), int(p[0][2:4]), int(p[0][4:6]), float(p[1]), int(p[2]), float(p[3]), float(p[4]), int(p[5]), group_height(int(p[2]))))

# Wykonanie agregacji na RDD
Początkowym etapem jest wyodrębnienie osobnych części RDD ze zbioru danych za pomocą operacji filter(). Do sprawdzenia warunku zadeklarowana prostę funkcję, która sprawdza czy wysokość mieści się w zadanym przedziale. Za pomocą operacji map() pobieramy pomiary ciśnienia atmosferycznego. Następnie za pomocą funkcji reduce() możemy policzyć całkowite ciśnienie na danej wysokości i podzielić tą sumę, aby otrzymać średnie ciśnienie.

## Za pomocą funkcji reduce()

In [5]:
# funkcja do filtrowania wysokosci
def h_is_near(h, hp, r):
  if h >= hp - r and h <= hp + r:
    return(True)
  else:
    return(False)

# wyodrebnienie danych RDD i pobranie cisnienia
pressure_near_ground = measures.filter(lambda x: h_is_near(int(x[4]), 100, 50)).map(lambda x: int(x[3]))
pressure_near_1km = measures.filter(lambda x: h_is_near(int(x[4]), 1000, 100)).map(lambda x: int(x[3]))
pressure_near_5km = measures.filter(lambda x: h_is_near(int(x[4]), 5000, 500)).map(lambda x: int(x[3]))
pressure_near_15km = measures.filter(lambda x: h_is_near(int(x[4]), 15000, 1500)).map(lambda x: int(x[3]))
pressure_near_30km = measures.filter(lambda x: h_is_near(int(x[4]), 30000, 3000)).map(lambda x: int(x[3]))

# obliczenie calkowitej liczby probek
near_ground_count = pressure_near_ground.count()
near_1km_count = pressure_near_1km.count()
near_5km_count = pressure_near_5km.count()
near_15km_count = pressure_near_15km.count()
near_30km_count = pressure_near_30km.count()

# agregacje
total_pressure_near_ground = pressure_near_ground.reduce(lambda x, y: x+y)
total_pressure_near_1km = pressure_near_1km.reduce(lambda x, y: x+y)
total_pressure_near_5km = pressure_near_5km.reduce(lambda x, y: x+y)
total_pressure_near_15km = pressure_near_15km.reduce(lambda x, y: x+y)
total_pressure_near_30km = pressure_near_30km.reduce(lambda x, y: x+y)

# wyniki
print("Average pressure near the ground:\t{}".format(total_pressure_near_ground/near_ground_count))
print("Average pressure at 1 km:\t{}".format(total_pressure_near_1km/near_1km_count))
print("Average pressure at 5 km:\t{}".format(total_pressure_near_5km/near_5km_count))
print("Average pressure at 15 km:\t{}".format(total_pressure_near_15km/near_15km_count))
print("Average pressure at 30 km:\t{}".format(total_pressure_near_30km/near_30km_count))

## Za pomocą funkcji aggregate()
Krótszy kod, wykonanie kilku operacji w każdej iteracji => mniej stages, znacznie szybciej

In [7]:
# funkcja do filtrowania wysokosci
def h_is_near(h, hp, r):
  if h >= hp - r and h <= hp + r:
    return(True)
  else:
    return(False)

# funkcja dodajaca wartosc do akumulatora
def add2acc(acc, value):
  return(acc[0] + value, acc[1] + 1)

# funkcja dodajaca dwa akumulatory
def addaccs(acc1, acc2):
  return(acc1[0] + acc2[0], acc1[1] + acc2[1])

# wyodrebnienie danych RDD i pobranie cisnienia
pressure_near_ground = measures.filter(lambda x: h_is_near(int(x[4]), 100, 50)).map(lambda x: int(x[3]))
pressure_near_1km = measures.filter(lambda x: h_is_near(int(x[4]), 1000, 100)).map(lambda x: int(x[3]))
pressure_near_5km = measures.filter(lambda x: h_is_near(int(x[4]), 5000, 500)).map(lambda x: int(x[3]))
pressure_near_15km = measures.filter(lambda x: h_is_near(int(x[4]), 15000, 1500)).map(lambda x: int(x[3]))
pressure_near_30km = measures.filter(lambda x: h_is_near(int(x[4]), 30000, 3000)).map(lambda x: int(x[3]))

# agregacje (calkowite cisnienie, calkowita liczba probek)
aggr_near_ground = pressure_near_ground.aggregate((0, 0), add2acc, addaccs)
aggr_near_1km = pressure_near_1km.aggregate((0, 0), add2acc, addaccs)
aggr_near_5km = pressure_near_5km.aggregate((0, 0), add2acc, addaccs)
aggr_near_15km = pressure_near_15km.aggregate((0, 0), add2acc, addaccs)
aggr_near_30km = pressure_near_30km.aggregate((0, 0), add2acc, addaccs)

# wyniki
print("Average pressure near the ground:\t{}".format(aggr_near_ground[0]/aggr_near_ground[1]))
print("Average pressure at 1 km:\t{}".format(aggr_near_1km[0]/aggr_near_1km[1]))
print("Average pressure at 5 km:\t{}".format(aggr_near_5km[0]/aggr_near_5km[1]))
print("Average pressure at 15 km:\t{}".format(aggr_near_15km[0]/aggr_near_15km[1]))
print("Average pressure at 30 km:\t{}".format(aggr_near_30km[0]/aggr_near_30km[1]))

# Przekształcenie RDD do DataFrame

In [9]:
# Hello World 
meteo_df = spark.createDataFrame(measures, ['year', 'month', 'day', 'pressure', 'height', 'temp', 'dewpoint', 'humidity', 'height_group'])
meteo_df.printSchema()
meteo_df.show()
# display(meteo_df)

# Wykonanie agregacji na DF
(DSL i SQL)

In [11]:
meteo_df.filter(meteo_df['height'] < 150).groupBy('year').count().show()
# test = meteo_df.map(lambda x: x if x['height'] > 100 else False)

# Wizualizacja danych

In [13]:
# Hello World

# Wykonanie dowolnego algorytmu z MLib

In [15]:
# Hello World